In [ ]:
import torch
from midi_model import MidiModel

In [ ]:
mh = MidiModel()
mh.init_data_dir()
mh.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training Parameters
mh.epochs = 50
mh.batch_size = 128
mh.learning_rate = 0.001

mh.sample_range_from = 0
mh.sample_range_to = 100

# LSTM Model Parameters
mh.input_size = 3
mh.hidden_size = 128
mh.output_size = 3

# LSTM Training Loss Parameters
mh.pitch_weight = 25.0
mh.step_weight = 15.0
mh.duration_weight = 10.0

In [ ]:
mh.load_train_data()

In [ ]:
mh.load_train_notes(mh.sample_range_from, mh.sample_range_to)

In [ ]:
mh.create_model()

In [ ]:
mh.train_model()

In [ ]:
input_notes = mh.midi.get_notes("../data/input.mid")
num_predictions = 10
seq_length = 4
temperature = 2.0

generated_notes, out_pm = mh.generate_notes(input_notes, num_predictions, seq_length, temperature)

mh.midi.display_midi("../data/generated/output.mid")

In [ ]:
mh.midi.display_audio(out_pm)